In [1]:
import os
import numpy as np
import torch
import pickle

from torch.utils.data import DataLoader
from generate_data import generate_vrp_data
from utils import load_model
from problems import CVRP

In [2]:
# model, _ = load_model('pretrained/cvrp_100/')
# model, _ = load_model('workstation_output/cvrp_50/20211106/epoch-99.pt')
model, _ = load_model('workstation_output/cvrp_50/20220217/epoch-21.pt')
torch.manual_seed(1234)
SIZE=100
num_samples=56
# dataset = CVRP.make_dataset(size=100, num_samples=1000)
dataset = CVRP.make_dataset(size=SIZE, num_samples=num_samples)

  [*] Loading model from workstation_output/cvrp_50/20220217/epoch-21.pt


In [3]:
# for i in range(100):
#     data_dice = {}
#     data_dice['X'] = [float(ele[0]) for ele in dataset[i]['loc']]
#     data_dice['X'] = [float(dataset[i]['depot'][0])] + data_dice['X'] ## add depot on top
    
#     data_dice['Y'] = [float(ele[1]) for ele in dataset[i]['loc']]
#     data_dice['Y'] = [float(dataset[i]['depot'][1])] + data_dice['Y'] ## add depot on top
    
#     data_dice['Demand'] = [0.0] + [float(ele) for ele in dataset[i]['demand']]
#     data_dice['start_time'] = [int(ele[0]) for ele in dataset[i]['time_window']]
#     data_dice['end_time'] = [int(ele[1]) for ele in dataset[i]['time_window']]
#     data_dice['service_time'] = [0.0 for ele in range(len(dataset[i]['time_window']))]
#     import pandas as pd
#     df = pd.DataFrame(data_dice)
#     df['Demand'] = df['Demand'] * 100
#     df.to_csv("~/fast-routing/benchmark/gurobi/data/node50/ex"+str(i)+".csv", index=False)

In [4]:
import pandas as pd
import os

index = 0
for data_path in os.listdir("solomon/csv/100"):
    df = pd.read_csv("solomon/csv/100/" + data_path).drop(columns = ["Unnamed: 0"])
    print("solomon/csv/100/" + data_path)
    print(df.head())

    df['XCOORD.'] = df['XCOORD.'] / 100
    df['YCOORD.'] = df['YCOORD.'] / 100
    df["DEMAND"] = df["DEMAND"] / 200
    df["READY TIME "] = df["READY TIME "] / 100
    df["DUE DATE "] = df["DUE DATE "] / 100

    dataset[index]['depot'] = torch.Tensor(df[['XCOORD.', 'YCOORD.']].values.tolist()[0])
    dataset[index]['loc'] = torch.Tensor(df[['XCOORD.', 'YCOORD.']].values.tolist()[1:])
    dataset[index]['demand'] = torch.Tensor(df["DEMAND"].values.tolist()[1:])
    dataset[index]['time_window'] = torch.Tensor(df[['READY TIME ', 'DUE DATE ']].values.tolist())
    index += 1

solomon/csv/100/r208.csv
   CUST NO.   XCOORD.  YCOORD.  DEMAND  READY TIME   DUE DATE   SERVICE TIME
0          0       35       35       0            0       1000             0
1          1       41       49      10            0        974            10
2          2       35       17       7            0        972            10
3          3       55       45      13            0        967            10
4          4       55       20      19          620        860            10
solomon/csv/100/c108.csv
   CUST NO.   XCOORD.  YCOORD.  DEMAND  READY TIME   DUE DATE   SERVICE TIME
0          0       40       50       0            0       1236             0
1          1       45       68      10          830       1049            90
2          2       45       70      30          756        939            90
3          3       42       66      10           16        336            90
4          4       42       68      10          643        866            90
solomon/csv/100/r103.csv
 

solomon/csv/100/r204.csv
   CUST NO.   XCOORD.  YCOORD.  DEMAND  READY TIME   DUE DATE   SERVICE TIME
0          0       35       35       0            0       1000             0
1          1       41       49      10            0        974            10
2          2       35       17       7            0        972            10
3          3       55       45      13            0        967            10
4          4       55       20      19          678        801            10
solomon/csv/100/rc106.csv
   CUST NO.   XCOORD.  YCOORD.  DEMAND  READY TIME   DUE DATE   SERVICE TIME
0          0       40       50       0            0        240             0
1          1       25       85      20          130        190            10
2          2       22       75      30           35         95            10
3          3       22       85      10           94        154            10
4          4       20       80      40          126        186            10
solomon/csv/100/c206.csv


In [5]:
# Need a dataloader to batch instances
dataloader = DataLoader(dataset, batch_size=1000)

# Make var works for dicts
batch = next(iter(dataloader))

# Run the model
model.eval()
model.set_decode_type('greedy')
with torch.no_grad():
    length, log_p, pi = model(batch, return_pi=True)
tours = pi

# Plot the results
# for i, (data, tour) in enumerate(zip(dataset, tours)):
#     fig, ax = plt.subplots(figsize=(10, 10))
#     plot_vehicle_routes(data, tour, ax, visualize_demands=False, demand_scale=50, round_demand=True)
#     fig.savefig(os.path.join('images', 'cvrp_{}.png'.format(i)))

---------------------
tensor([[ 0.4104],
        [ 3.7000],
        [ 0.5000],
        [ 9.6700],
        [ 1.2700],
        [ 1.2300],
        [ 2.1400],
        [ 3.4500],
        [ 1.4900],
        [ 3.2200],
        [ 1.8600],
        [ 3.2200],
        [ 1.0700],
        [ 2.0300],
        [ 1.2400],
        [ 1.0200],
        [ 2.8600],
        [ 0.2110],
        [ 0.8000],
        [ 8.2500],
        [ 9.6700],
        [10.6500],
        [ 1.1400],
        [ 0.5000],
        [ 1.7800],
        [ 2.0300],
        [10.6500],
        [ 1.9000],
        [ 4.2500],
        [ 0.2110],
        [ 1.7500],
        [ 1.2400],
        [11.5400],
        [ 2.4600],
        [ 0.1803],
        [ 3.3500],
        [ 2.5200],
        [ 3.2400],
        [ 2.0700],
        [ 0.9000],
        [ 2.0300],
        [ 1.7300],
        [ 1.7800],
        [ 3.8400],
        [ 1.3100],
        [ 1.7800],
        [ 1.0500],
        [ 0.4000],
        [ 0.7900],
        [ 9.8500],
        [ 3.2200],
        [

---------------------
tensor([[ 2.4881],
        [ 4.5645],
        [ 1.7841],
        [23.2100],
        [ 2.1275],
        [ 2.3017],
        [ 8.9813],
        [ 0.9522],
        [ 0.3547],
        [ 4.5936],
        [ 2.6900],
        [ 5.1900],
        [ 2.1513],
        [ 3.5300],
        [ 1.8954],
        [ 1.4131],
        [ 6.9293],
        [ 1.7282],
        [ 0.8145],
        [ 6.3100],
        [ 5.2469],
        [16.4100],
        [ 1.8138],
        [ 1.9905],
        [ 0.8600],
        [ 1.2631],
        [ 0.5571],
        [ 5.8500],
        [ 0.6329],
        [ 2.3051],
        [ 2.7019],
        [ 1.8456],
        [ 6.8834],
        [ 5.5000],
        [ 2.1674],
        [ 4.1600],
        [ 1.0606],
        [ 6.7357],
        [ 5.4129],
        [ 1.9100],
        [ 3.0441],
        [ 0.6700],
        [ 2.5856],
        [ 9.2758],
        [ 1.9554],
        [ 0.9500],
        [ 2.0026],
        [ 1.8909],
        [ 2.2149],
        [29.2100],
        [ 7.1600],
        [

---------------------
tensor([[ 2.6073],
        [ 0.1612],
        [ 1.8908],
        [ 0.5771],
        [ 0.3905],
        [ 1.1403],
        [ 0.5656],
        [ 5.1931],
        [ 4.8816],
        [ 0.2915],
        [ 6.6200],
        [ 2.4286],
        [ 2.2085],
        [ 1.6900],
        [ 1.5456],
        [ 0.9711],
        [ 3.5900],
        [ 0.8609],
        [ 2.8959],
        [ 0.1513],
        [ 2.1836],
        [ 6.0932],
        [ 1.8777],
        [ 0.9331],
        [ 0.9601],
        [ 1.0528],
        [ 1.7528],
        [ 0.1838],
        [ 1.2700],
        [ 2.0639],
        [ 0.3227],
        [ 0.7561],
        [ 2.4600],
        [ 2.2100],
        [ 0.5900],
        [ 0.7639],
        [ 4.8098],
        [ 2.5074],
        [ 4.4300],
        [ 0.1476],
        [ 1.1216],
        [ 2.0008],
        [ 2.1148],
        [ 1.0060],
        [ 6.0417],
        [ 1.8081],
        [ 1.7456],
        [ 0.2915],
        [ 0.9300],
        [ 3.6706],
        [ 3.5000],
        [

tensor([[0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [1.0141],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.6500],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [1.2402],
        [0.0000],
        [0.0000],
        [0.0000],
        [1.2402],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0

In [6]:
##CREATE DATA FOR GUROBI##############################################    
# for i in range(num_samples):
#     data_dice = {}
#     data_dice['X'] = [float(ele[0]) for ele in model.state.coords[i]]
#     data_dice['Y'] = [float(ele[1]) for ele in model.state.coords[i]]
    
#     data_dice['Demand'] = [0.0] + [float(ele) for ele in model.state.demand[i]]
#     data_dice['start_time'] = [int(ele[0]) for ele in model.state.time_window[i]]
#     data_dice['end_time'] = [int(ele[1]) for ele in model.state.time_window[i]]
#     data_dice['service_time'] = [0.0 for ele in range(len(model.state.time_window[i]))]
#     import pandas as pd
#     df = pd.DataFrame(data_dice)
#     df['Demand'] = df['Demand'] * 100
#     df.to_csv("~/fast-routing/benchmark/gurobi/data/node" + str(SIZE) +"/ex"+str(i)+".csv", index=False)


# def make_data_model(model, index):
#     data_model = {}
#     data_model['time_matrix'] = []
#     size = len(model.state.coords[index])
#     coords = np.array(model.state.coords[index])
#     for i in range(size):
#         temp = []
#         for j in range(size):
#             temp.append(round(np.linalg.norm(coords[i]-coords[j]) * 1000))
#         data_model['time_matrix'].append(temp)

#     data_model['time_windows'] = []
#     time_window = model.state.time_window[index]
#     for i in range(size):
#         data_model['time_windows'].append((int(time_window[i][0])*1000, int(time_window[i][1])*1000))

#     data_model['demands'] = [0] + [round(float(demand)*1000) for demand in model.state.demand[index]]
#     data_model['num_vehicles'] = 100
#     data_model['vehicle_capacities'] = [1000] * data_model['num_vehicles']
#     data_model['depot'] = 0
#     return data_model

# for i in range(num_samples):
#     data = make_data_model(model, i)
#     with open("/home/asaki/fast-routing/benchmark/ortools/data/node" + str(SIZE) +"/ex"+str(i)+".pickle", mode="wb") as f:
#         pickle.dump(data, f)
#     with open("/home/asaki/fast-routing/benchmark/ortools_guided/data/node" + str(SIZE) +"/ex"+str(i)+".pickle", mode="wb") as f:
#         pickle.dump(data, f)

In [7]:
##### atteintion solomon result ##########
def to_depot_distance(coords, location, i):
    return np.linalg.norm(np.array(coords[location[i-1]]) * 100 - np.array(coords[location[i]]) * 100)

total_list = []
for instance_index in range(56):
    print(instance_index)
    total = 0
    for i in range(len(model.state.current_time_list)):
        if instance_index == 0 and True:
            print(model.state.current_time_list[i][instance_index] * 100, dataset.data[instance_index]['time_window'][pi[instance_index][i]] * 100)
#             print((model.state.coords[instance_index][pi[instance_index][i]] - model.state.coords[instance_index][pi[instance_index][i+1]]).norm(p=2, dim=-1))
        current = float(model.state.current_time_list[i][instance_index] * 100)
        if i != 0 and current == 0.0:
#             print(to_depot_distance(model.state.coords[instance_index], tours[instance_index], i))
            total += previous + to_depot_distance(model.state.coords[instance_index], tours[instance_index], i)
        previous = current
    print(total)
    total_list.append(total)

0
tensor([51.0366]) tensor([  0.0000, 947.9999])
tensor([73.0782]) tensor([  0.0000, 939.9999])
tensor([339.]) tensor([329., 569.])
tensor([361.1655]) tensor([307., 547.])
tensor([415.3130]) tensor([228.0000, 468.0000])
tensor([487.0544]) tensor([354., 594.])
tensor([509.0959]) tensor([323., 563.])
tensor([575.7440]) tensor([560., 800.])
tensor([630.]) tensor([620.0000, 860.0001])
tensor([651.4017]) tensor([622., 862.])
tensor([681.4017]) tensor([639., 879.])
tensor([740.]) tensor([730., 970.])
tensor([775.9423]) tensor([716.0000, 956.0001])
tensor([815.5565]) tensor([740., 980.])
tensor([0.]) tensor([   0., 1000.])
tensor([49.6611]) tensor([  0., 950.])
tensor([76.1535]) tensor([  0., 958.])
tensor([91.1535]) tensor([  0., 953.])
tensor([121.1535]) tensor([  0.0000, 960.0001])
tensor([138.2246]) tensor([  0.0000, 960.0001])
tensor([157.6585]) tensor([  0., 968.])
tensor([182.9556]) tensor([  0.0000, 964.9999])
tensor([200.1667]) tensor([  0., 963.])
tensor([218.2290]) tensor([  0., 95

In [8]:
# for i, (data, tour) in enumerate(zip(dataset, tours)):
#     fig, ax = plt.subplots(figsize=(10, 10))
#     plot_vehicle_routes(data, tour, ax, visualize_demands=False, demand_scale=2010, round_demand=True)
#     # fig.savefig(os.path.join('images', 'cvrp_{}.png'.format(i)))

In [9]:
# with open("result_"+str(SIZE)+".pickle", mode="wb") as f:
#     pickle.dump(total_list, f)
# total_list

In [10]:
sum(total_list)

416414.3250513077

In [19]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def make_data_model(model, index):
    global data_model
    data_model = {}
    data_model['time_matrix'] = []
    size = len(model.state.coords[index])
    coords = np.array(model.state.coords[index])
    for i in range(size):
        temp = []
        for j in range(size):
            if i != 0:
                temp.append(round(np.linalg.norm(coords[i]-coords[j]) * 1000) + 100)
            else:
                temp.append(round(np.linalg.norm(coords[i]-coords[j]) * 1000))
            
        data_model['time_matrix'].append(temp)

    data_model['time_windows'] = []
    time_window = model.state.time_window[index]
    for i in range(size):
        data_model['time_windows'].append((int(time_window[i][0]*1000), int(time_window[i][1]*1000)))
        
    data_model['demands'] = [0] + [round(float(demand)*200) for demand in model.state.demand[index]]
    data_model['num_vehicles'] = 100
    data_model['vehicle_capacities'] = [200] * data_model['num_vehicles']
    data_model['depot'] = 0
    return data_model

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    solution_list = []
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        temp = []
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index),
                solution.Min(time_var),
                solution.Max(time_var)
            )
            temp.append(
                [manager.IndexToNode(index),
                 solution.Min(time_var)/10,
                 solution.Max(time_var)/10]
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        temp.append(
                [manager.IndexToNode(index),
                 solution.Min(time_var)/10,
                 solution.Max(time_var)/10]
            )
        if len(temp) != 1:
            solution_list.append(temp)
        total_time += solution.Min(time_var)
#     print('Total time of all routes: {}min'.format(total_time))
    return solution_list


def main(model, index):
    """Solve the VRP with time windows."""
    global data
    # Instantiate the data problem.
#     data = create_data_model()
    data = make_data_model(model, index)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        100000,  # allow waiting time
        100000,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

    # Add time window constraints for each vehicle start node.
    depot_idx = data['depot']
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])
        
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
#     search_parameters.local_search_metaheuristic = (
#         routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
#     search_parameters.time_limit.seconds = 30
#     search_parameters.log_search = True

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        total_time = print_solution(data, manager, routing, solution)
    return total_time


def ortools(model, index):
    return main(model, index)

totaltotal = 0
for i in range(56):
    solution_list = ortools(model, i)
    tot = 0
    for sol in solution_list:
        tot += sol[-1][2]
    print(tot)
    totaltotal += tot


4906.599999999999
7580.2
2912.9
22114.499999999996
2534.9000000000005
2421.3
7173.9
8251.2
7963.8
5583.3
6099.200000000001
5649.1
2615.0
7686.000000000001
3732.9
2584.1
8655.8
2301.3
7713.6
18858.6
21989.8
24847.8
2930.399999999999
3393.099999999999
3354.5
6285.5
14783.199999999999
6983.799999999999
7935.500000000001
2383.0
5778.299999999999
2751.6000000000004
21978.2
8155.900000000001
2689.7
7134.5
7297.599999999999
5824.2
4849.8
5079.2
8035.4
3363.0
2752.3000000000006
7979.799999999999
5736.4
3358.399999999999
2552.2000000000003
2822.4999999999995
2217.3
23577.399999999998
5627.6
2521.1
8294.8
21088.300000000003
5158.699999999999
5584.599999999999


In [22]:
solution_list

[[[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0, 0.0], [0, 0.0, 0.0]],
 [[0, 0.0,

In [27]:
data['time_matrix'][10][0]

355.0

In [28]:
25.5**2

650.25

In [24]:
5**2 + 25**2

650

In [23]:
pd.read_csv("solomon/csv/100/r209.csv").drop(columns = ["Unnamed: 0"]).head(50)

,CUST NO.,XCOORD.,YCOORD.,DEMAND,READY TIME,DUE DATE,SERVICE TIME
0,0,35,35,0,0,1000,0
1,1,41,49,10,636,919,10
2,2,35,17,7,74,351,10
3,3,55,45,13,498,613,10
4,4,55,20,19,470,965,10
5,5,15,30,26,20,370,10
6,6,25,30,3,266,663,10
7,7,20,50,5,292,449,10
8,8,10,43,9,288,597,10
9,9,55,60,16,254,643,10


In [15]:
36+14*14

232

In [13]:
data

{'time_matrix': [[0.0,
   152.0,
   180.0,
   224.0,
   250.0,
   206.0,
   112.0,
   212.0,
   262.0,
   320.0,
   255.0,
   335.0,
   150.0,
   112.0,
   320.0,
   304.0,
   292.0,
   304.0,
   158.0,
   320.0,
   316.0,
   180.0,
   269.0,
   361.0,
   300.0,
   335.0,
   112.0,
   50.0,
   63.0,
   298.0,
   255.0,
   175.0,
   340.0,
   248.0,
   361.0,
   410.0,
   414.0,
   212.0,
   424.0,
   340.0,
   112.0,
   289.0,
   255.0,
   342.0,
   319.0,
   292.0,
   355.0,
   342.0,
   278.0,
   439.0,
   170.0,
   269.0,
   113.0,
   45.0,
   228.0,
   305.0,
   292.0,
   232.0,
   91.0,
   178.0,
   180.0,
   255.0,
   255.0,
   349.0,
   465.0,
   499.0,
   405.0,
   439.0,
   214.0,
   122.0,
   211.0,
   397.0,
   225.0,
   201.0,
   246.0,
   278.0,
   157.0,
   197.0,
   311.0,
   256.0,
   211.0,
   276.0,
   233.0,
   211.0,
   243.0,
   230.0,
   354.0,
   184.0,
   192.0,
   90.0,
   322.0,
   256.0,
   184.0,
   202.0,
   120.0,
   149.0,
   153.0,
   172.0,
   213.0,
  

In [ ]:
totaltotal = 0
for i in range(100):
    solution_list = ortools(model, i)
    tot = 0
    for sol in solution_list:
        tot += sol[-1][2]
    print(tot)
    totaltotal += tot

38.684
65.571


In [ ]:
totaltotal

In [ ]:
first step  adding time concept
second step refactor existing code
third step  modify endode layer (more practical one)
fourth step add time window as an instance
